In [189]:
import pandas as pd
from math import sqrt
import random

# Importing Datasets

In [190]:
# World Cup Matches
matches = pd.read_csv("../WorldCupMatches.csv")

# World Cup Players
players = pd.read_csv("../WorldCupPlayers.csv")

# World Cups
cups = pd.read_csv("../WorldCups.csv")

# Player Stats
stats = pd.read_csv("../PlayersStats.csv")

# Match Players with Matches

# Add Player's overall FIFA score

### Define `final` DataFrame

In [191]:
columns = list(matches.columns)

new = [
    "Home Coach Name",
    "Home Player 1", "Home Player 2", 
    "Home Player 3", "Home Player 4", 
    "Home Player 5", "Home Player 6", 
    "Home Player 7", "Home Player 8", 
    "Home Player 9", "Home Player 10",
    "Home Player 11",

    "Away Coach Name",
    "Away Player 1", "Away Player 2", 
    "Away Player 3", "Away Player 4", 
    "Away Player 5", "Away Player 6", 
    "Away Player 7", "Away Player 8", 
    "Away Player 9","Away Player 10",
    "Away Player 11",
    
    "Home Player 1 Stats", "Home Player 2 Stats", 
    "Home Player 3 Stats", "Home Player 4 Stats", 
    "Home Player 5 Stats", "Home Player 6 Stats", 
    "Home Player 7 Stats", "Home Player 8 Stats", 
    "Home Player 9 Stats","Home Player 10 Stats",
    "Home Player 11 Stats",

    "Away Player 1 Stats", "Away Player 2 Stats", 
    "Away Player 3 Stats", "Away Player 4 Stats", 
    "Away Player 5 Stats", "Away Player 6 Stats", 
    "Away Player 7 Stats", "Away Player 8 Stats", 
    "Away Player 9 Stats","Away Player 10 Stats",
    "Away Player 11 Stats"
]

columns += new
final = pd.DataFrame(columns=columns)

### Preparing utility functions

In [192]:
# final.columns

In [193]:
stats = stats.sort_values(by="Name")
stats = stats.reset_index().drop("index", axis=1)

In [194]:
stats.head()

,Name,Overall
0,A. Abbas,62
1,A. Abbas,64
2,A. Abdallah,52
3,A. Abdennour,77
4,A. Abdi,71


In [195]:
def clean(x):
    x = x.lower().strip().split(" ")

    if len(x) == 1:
        x = x[0]

    elif len(x) == 2:
        if "." in x[0]:
            x = x[1]
        elif "." in x[1]:
            x = x[0]
        else:
            x = x[0] + x[1]

    elif len(x) == 3:
        if "." in x[0]:
            x = x[1] + x[2]
        elif "." in x[1]:
            x = x[0] + x[2]
        elif "." in x[2]:
            x = x[0] + x[1]
        else:
            x = x[0] + x[1] + x[2]

    elif len(x) == 4:
        if "(" in x[3] or ")" in x[3]:
            if "." in x[0]:
                x = x[1] + x[2]
            elif "." in x[1]:
                x = x[0] + x[2]
            elif "." in x[2]:
                x = x[0] + x[1]
            else:
                x = x[0] + x[1] + x[2]
        else:
            if "." in x[0]:
                x = x[1] + x[2] + x[3]
            elif "." in x[1]:
                x = x[0] + x[2] + x[3]
            elif "." in x[2]:
                x = x[0] + x[1] + x[3]
            elif "." in x[3]:
                x = x[0] + x[1] + x[2]
            else:
                x = x[0] + x[1] + x[2] + x[3]
            
    else:
        pass
    
    return x

def find_start_index(name):
    name = clean(name)
    i = 0
    jump_step = int(sqrt(len(stats)))
    
    while i < len(stats) and name[0] > clean(stats.iloc[i]["Name"])[0]:
        i += jump_step
    
    i -= jump_step
    return i

def find_end_index(name):
    name = clean(name)
    i = len(stats)-1
    jump_step = int(sqrt(len(stats)))
    
    while i >= 0 and name[0] < clean(stats.iloc[i]["Name"])[0]:
        i -= jump_step
    
    i += jump_step
    return i

def similarity_score(a, b):
    a = clean(a)
    b = clean(b)
    count = 0
    length = min(len(a), len(b))
    
    for i in range(length):
        if a[i] == b[i]:
            count+=1
            
    return count/length

### Define `merging`

In [196]:
DEFAULT_OVERALL_SCORE = 65

def run_merging(debug=False, DEFAULT_OVERALL_SCORE=DEFAULT_OVERALL_SCORE):
    global final
    
    for i in range(len(matches)):
        
        # # Add Players's name # #
        
        home = ["coach"]
        away = ["coach"]
        HomeInitials = matches.iloc[i]["Home Team Initials"]
        AwayInitials = matches.iloc[i]["Away Team Initials"]   

        for j in range(len(players)):
            player = players.iloc[j]["Player Name"]
            coach = players.iloc[j]["Coach Name"]

            if HomeInitials == players.iloc[j]["Team Initials"]:
                home[0] = coach
                home.append(player)
            elif AwayInitials == players.iloc[j]["Team Initials"]:
                away[0] = coach
                away.append(player)
            else:
                pass

        while len(home) != 12:
            if len(home) > 12:
                del home[random.randint(1, len(home) - 1)]
            else:
                home.append(random.choice(home))


        while len(away) != 12:
            if len(away) > 12:
                del away[random.randint(1, len(away) - 1)]
            else:
                away.append(random.choice(away))

        # # Add Player's overall FIFA score # #
        
        home_stats = []
        for name in home[1:]:
            best_match_index = -1
            best_score = 0

            for j in range(find_start_index(name), find_end_index(name)):         
                current_score = similarity_score(name, stats.iloc[j]["Name"])

                if current_score > best_score:
                    best_score = current_score
                    best_match_index = j
            
            if best_match_index != -1:
                home_stats.append(stats.iloc[best_match_index]["Overall"])
                if debug:
                    print("Search: ", name, " | ", "Best match: ", stats.iloc[best_match_index]["Name"])
            else:
                home_stats.append(DEFAULT_OVERALL_SCORE)
                        
        away_stats = []
        for name in away[1:]:
            best_match_index = -1
            best_score = 0

            for j in range(find_start_index(name), find_end_index(name)):            
                current_score = similarity_score(name, stats.iloc[j]["Name"])

                if current_score > best_score:
                    best_score = current_score
                    best_match_index = j
            
            if best_match_index != -1:
                away_stats.append(stats.iloc[best_match_index]["Overall"])
                if debug:
                    print("Search: ", name, " | ", "Best match: ", stats.iloc[best_match_index]["Name"])
            else:
                away_stats.append(DEFAULT_OVERALL_SCORE)
                    
        data = list(matches.iloc[i])
        data += home
        data += away
        data += home_stats
        data += away_stats

        final = final.append(pd.Series(data, index=columns), ignore_index=True)

### Run `merging` (will take about an hour)

In [197]:
run_merging(debug=False)

Search:  Lucas RADEBE  |  Best match:  Lucas
Search:  RADEBE  |  Best match:  T. Hadebe
Search:  KHUNE  |  Best match:  I. Khune
Search:  JOSEPHS  |  Best match:  A. Rose
Search:  JOSEPHS  |  Best match:  A. Rose
Search:  JOSEPHS  |  Best match:  A. Rose
Search:  JOSEPHS  |  Best match:  A. Rose
Search:  JOSEPHS  |  Best match:  A. Rose
Search:  RADEBE  |  Best match:  T. Hadebe
Search:  JOSEPHS  |  Best match:  A. Rose
Search:  Lucas RADEBE  |  Best match:  Lucas
Search:  Horacio CASARIN  |  Best match:  I. Hora
Search:  Gustavo PENA  |  Best match:  Gustavo Hebling
Search:  Arturo VAZQUEZ  |  Best match:  Artur
Search:  Jose REYES  |  Best match:  Josete
Search:  Pedro SOTO  |  Best match:  Pedro
Search:  Tomas BOY  |  Best match:  X. Tomas
Search:  Alberto GARCIA ASPE  |  Best match:  Alberto
Search:  O. PEREZ  |  Best match:  M. Perez
Search:  R. MARQUEZ  |  Best match:  R. Marquez
Search:  G TORRADO  |  Best match:  F. Iturrado


KeyboardInterrupt: 

In [198]:
final.head()

,Year,Datetime,Stage,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Attendance,Half-time Home Goals,Half-time Away Goals,...,Away Player 2 Stats,Away Player 3 Stats,Away Player 4 Stats,Away Player 5 Stats,Away Player 6 Stats,Away Player 7 Stats,Away Player 8 Stats,Away Player 9 Stats,Away Player 10 Stats,Away Player 11 Stats


# Save final into CSV file

In [199]:
final.to_csv("../final.csv", index=0)